In [1]:
import pandas as pd
import urllib.request
import urllib.parse

In [4]:
course_url_endpoint = 'http://apis.data.go.kr/B552881/kmooc/courseList?'
detail_url_endpoint = 'http://apis.data.go.kr/B552881/kmooc/courseDetail?'

page = 1
id = 'course-v1:AIIA+AIIA02+2021_T2_AIIA02'
catalog_params = {
    'ServiceKey' : 'NhRuAoFGxRza0hrpsOJ1%2Fnvh6wrZSUse31YxriXHw273mOglft9Jy2u0JtJhzWqJ4vFUOHbaBWz0ObbzWR1lqw%3D%3D',
    'page' : page
}

detail_params = {
    'ServiceKey' : 'NhRuAoFGxRza0hrpsOJ1%2Fnvh6wrZSUse31YxriXHw273mOglft9Jy2u0JtJhzWqJ4vFUOHbaBWz0ObbzWR1lqw%3D%3D',
    'CourseId' : id
}

## requests.get이 url을 encoding하기 때문에 이중 encoding을 막기 위한 작업이 필요

from urllib.parse import unquote

catalog_params['ServiceKey'] = unquote(catalog_params['ServiceKey'])
detail_params['ServiceKey'] = unquote(catalog_params['ServiceKey'])

In [45]:
course_resp.json() ## json 형태의 파일로 가져옴

{'pagination': {'count': 1126,
  'previous': None,
  'num_pages': 113,
  'next': 'http://www.kmooc.kr/api/courses/v1/course/list/?SG_APIM=2ug8Dm9qNBfD32JLZGPN64f3EoTlkpD8kSOHWfXpyrY&ServiceKey=NhRuAoFGxRza0hrpsOJ1%2Fnvh6wrZSUse31YxriXHw273mOglft9Jy2u0JtJhzWqJ4vFUOHbaBWz0ObbzWR1lqw%3D%3D&page=2'},
 'results': [{'blocks_url': 'http://www.kmooc.kr/api/courses/v1/blocks/?course_id=course-v1%3AACRCEDU%2BACRC01%2B2020_03',
   'effort': '00:15@07#01:40$07:00',
   'end': '2020-12-15T15:00:00Z',
   'enrollment_start': '2020-11-05T00:00:00Z',
   'enrollment_end': '2020-12-15T15:00:00Z',
   'id': 'course-v1:ACRCEDU+ACRC01+2020_03',
   'media': {'course_image': {'uri': '/asset-v1:ACRCEDU+ACRC01+2020_03+type@asset+block@1022_청렴_저용량__썸네일.png'},
    'course_video': {'uri': None},
    'image': {'raw': 'http://www.kmooc.kr/asset-v1:ACRCEDU+ACRC01+2020_03+type@asset+block@1022_%EC%B2%AD%EB%A0%B4_%EC%A0%80%EC%9A%A9%EB%9F%89__%EC%8D%B8%EB%84%A4%EC%9D%BC.png',
     'small': 'http://www.kmooc.kr/asset-v1:AC

들어있는 값을 확인해보면.. next가 null이면 그만 가져오면 됨.
전체 count를 센 다음에 돌려가면서 count가 바뀔 때 새로운 값들만큼 가져오면 됨.


In [13]:
course_json = course_resp.json()

json파일의 형태로 데이터를 저장. 
전체 json은 pagination(dictionary 형식으로 저장되어 있음), results (dictionary가 list형식으로 한번 더 묶여있음)


In [15]:
page

[{'count': 1126,
  'previous': None,
  'num_pages': 113,
  'next': 'http://www.kmooc.kr/api/courses/v1/course/list/?SG_APIM=2ug8Dm9qNBfD32JLZGPN64f3EoTlkpD8kSOHWfXpyrY&ServiceKey=NhRuAoFGxRza0hrpsOJ1%2Fnvh6wrZSUse31YxriXHw273mOglft9Jy2u0JtJhzWqJ4vFUOHbaBWz0ObbzWR1lqw%3D%3D&page=2'}]

In [18]:
page[0]['count']

1126

next로 넘어가고 next가 없으면 거기서 멈추면 됨
새로운 데이터 넣을 때는 count와 비교해서 전체 count수보다 더 많으면 새로운 개수까지만 넣으면 됨. 

# 전체 course list를 crawling하기

In [2]:
# crawling 해야할 다음 페이지 주소 : [course_json['pagination']][0]['next'] 
# next 값이 없으면 stop
# 굳이 next로 안하고 page 수에 맞게 다음페이지로 넘어가는 게 더 간편하지 않을까? 그러면 호출을 딱 한번만 해도 되는데;..
# next로 넘어가야 하는 이유는 크롤링 하는 도중에 새로운 데이터가 넘어와서 페이지가 늘어나고, 그러면 끝단에 있는 데이터들을 못 끌어올 수 있으니까
# 최초에 db를 만들 때는 next로 넘어가고 다음부터는 첫 페이지의 페이지 수만 체크하면서 db에 중복데이터가 생겨나지 않을 때까지 크롤링하면 됨

## import packages
import pandas as pd
import requests
from urllib.parse import unquote
from pandas.io.json import json_normalize
import time


###############################################
###############################################
## api address
course_url_endpoint = 'http://apis.data.go.kr/B552881/kmooc/courseList?'
## starting page (page = 1)
first_url = 'http://apis.data.go.kr/B552881/kmooc/courseList?&ServiceKey=NhRuAoFGxRza0hrpsOJ1%2Fnvh6wrZSUse31YxriXHw273mOglft9Jy2u0JtJhzWqJ4vFUOHbaBWz0ObbzWR1lqw%3D%3D&page=1'


##############################################
### define function to fetch response ###
def fetch_response(url, df):
    course_resp = requests.get(url)
    course_json = course_resp.json()
    pagination = course_json['pagination'] #type(page) = dict
    course_df = pd.json_normalize(course_json['results'])  #type(course_df) = dataframe
    df = pd.concat([df,course_df]) ## concatenate newly crawled data into a single dataframe
    return pagination, df

#############################################
df = pd.DataFrame()
pagination, df = fetch_response(first_url, df)## crawling the first page


while True:  
    if pagination['next'] == None:  ## The last page doesn't have 'next' key value
        break;
    else:
        pagination, df = fetch_response(pagination['next'], df)
        time.sleep(3) ## 3초 미만은 서버 문제로 오류 발생함

In [30]:
df.head()

,blocks_url,effort,end,enrollment_start,enrollment_end,id,name,number,org,short_description,...,video_time,week,learning_time,preview_video,course_id,media.course_image.uri,media.course_video.uri,media.image.raw,media.image.small,media.image.large
0,http://www.kmooc.kr/api/courses/v1/blocks/?cou...,00:15@07#01:40$07:00,2020-12-15T15:00:00Z,2020-11-05T00:00:00Z,2020-12-15T15:00:00Z,course-v1:ACRCEDU+ACRC01+2020_03,문화와 생활 속 청렴,ACRC01,ACRCEDU,문화와 생활 속 청렴강좌는 인문학 속 역사 이야기와 생활 속 청렴 정보통으로 구성되...,...,01:40,07,00:15,,course-v1:ACRCEDU+ACRC01+2020_03,/asset-v1:ACRCEDU+ACRC01+2020_03+type@asset+bl...,None,http://www.kmooc.kr/asset-v1:ACRCEDU+ACRC01+20...,http://www.kmooc.kr/asset-v1:ACRCEDU+ACRC01+20...,http://www.kmooc.kr/asset-v1:ACRCEDU+ACRC01+20...
1,http://www.kmooc.kr/api/courses/v1/blocks/?cou...,00:40@15#10:00$10:00,2021-07-31T23:30:00Z,2021-05-01T00:00:00Z,2021-05-31T23:30:00Z,course-v1:AIIA+AIIA01+2021_T2_AIIA01,확률적 그래픽 모델,AIIA01,AIIA,"본 강좌의 목적은 확률적 그래픽 모델의 개요를 이해하고, 이를 응용한 각종 딥러닝 ...",...,10:00,15,00:40,,course-v1:AIIA+AIIA01+2021_T2_AIIA01,/asset-v1:AIIA+AIIA01+2021_T2_AIIA01+type@asse...,http://www.youtube.com/watch?v=JWggZMvQNus,http://www.kmooc.kr/asset-v1:AIIA+AIIA01+2021_...,http://www.kmooc.kr/asset-v1:AIIA+AIIA01+2021_...,http://www.kmooc.kr/asset-v1:AIIA+AIIA01+2021_...
2,http://www.kmooc.kr/api/courses/v1/blocks/?cou...,00:30@15#06:50$06:00,2021-07-31T00:00:00Z,2021-05-01T00:00:00Z,2021-05-31T23:30:00Z,course-v1:AIIA+AIIA02+2021_T2_AIIA02,전산인지과학 및 인간지능과 인공지능,AIIA02,AIIA,본 강좌는 전산 인지과학에 대한 범위와 요소 그리고 지능 정의와 측정에 대한 다양한...,...,06:50,15,00:30,,course-v1:AIIA+AIIA02+2021_T2_AIIA02,/asset-v1:AIIA+AIIA02+2021_T2_AIIA02+type@asse...,None,http://www.kmooc.kr/asset-v1:AIIA+AIIA02+2021_...,http://www.kmooc.kr/asset-v1:AIIA+AIIA02+2021_...,http://www.kmooc.kr/asset-v1:AIIA+AIIA02+2021_...
3,http://www.kmooc.kr/api/courses/v1/blocks/?cou...,01:00@06#03:31$05:15,2021-05-12T00:00:00Z,2021-03-23T00:00:00Z,2021-03-31T11:59:00Z,course-v1:AJOU+AJOU01+2021_T1,인간의 판단과 의사결정,AJOU01,AJOU,인간의 판단과 의사결정 강의에 오신것을 환영합니다.\n인지심리학자 김경일 교수와 함...,...,03:31,06,01:00,,course-v1:AJOU+AJOU01+2021_T1,/asset-v1:AJOU+AJOU01+2021_T1+type@asset+block...,None,http://www.kmooc.kr/asset-v1:AJOU+AJOU01+2021_...,http://www.kmooc.kr/asset-v1:AJOU+AJOU01+2021_...,http://www.kmooc.kr/asset-v1:AJOU+AJOU01+2021_...
4,http://www.kmooc.kr/api/courses/v1/blocks/?cou...,01:30@15#09:00$27:20,2020-12-31T00:00:00Z,2020-10-12T00:00:00Z,2020-12-31T00:00:00Z,course-v1:ANU+ANU01+2020_T3,정보통신 보안,ANU01,ANU,"본 강좌의 목적은 정보통신 보안 일반(암호화, 전자서명, 악성코드)와 네트워크, 서...",...,09:00,15,01:30,,course-v1:ANU+ANU01+2020_T3,/asset-v1:ANU+ANU01+2020_T3+type@asset+block@메...,http://www.youtube.com/watch?v=6zIS6nagiY0,http://www.kmooc.kr/asset-v1:ANU+ANU01+2020_T3...,http://www.kmooc.kr/asset-v1:ANU+ANU01+2020_T3...,http://www.kmooc.kr/asset-v1:ANU+ANU01+2020_T3...


In [15]:
df.to_csv('course_df.csv', index = False)

## crawling course detail


In [7]:
## parameter 살리고 긁어오기
## course detail은 id를 url parameter로 넣어야'만' 조회 가능
## course list에 있는 id값을 key로 두고 course detail json 파일의 핵심 값들을 dataframe에 넣기

## import packages
import pandas as pd
import requests
from urllib.parse import unquote
from pandas.io.json import json_normalize
import time
import tqdm


detail_url_endpoint = 'http://apis.data.go.kr/B552881/kmooc/courseDetail?'
detail_params = {
    'ServiceKey' : 'NhRuAoFGxRza0hrpsOJ1%2Fnvh6wrZSUse31YxriXHw273mOglft9Jy2u0JtJhzWqJ4vFUOHbaBWz0ObbzWR1lqw%3D%3D'
    # 'CourseId' : id     will be defined in fuction
}
detail_params['ServiceKey'] = unquote(detail_params['ServiceKey'])

## org_name , classfy_name, middle_classfy_name, overview -> html형식으로 가져와야 함.
column_list = ['teachers','org_name', 'classfy_name', 'middle_classfy_name', 'overview']
info_df = pd.DataFrame(df.id)   ## info_df는 최초 크롤링 데이터베이스



##########################################################
### function to crawl course detail for each course id ###
def fetch_course_detail(idd, column_list, params: dict, df):
    ##필요한 column_list는 바뀔 수 있기 때문에 함수값으로 받도록 함
    ## fetch course detail response corresponding to course id
    params['CourseId'] = idd
    course_json = requests.get(detail_url_endpoint, params = params).json()
    for col in column_list:
        df.loc[df['id']==idd, col] = course_json[col]
    return df


##########################################################
### fetch response to each row ###

for idd in tqdm.tqdm(info_df['id'].unique().tolist()):
    info_df = fetch_course_detail(idd, column_list, detail_params, info_df)
    time.sleep(3)



100%|████████████████████████████████████████████████████████████████████████████| 1132/1132 [1:04:34<00:00,  3.42s/it]


"\nwith tqdm.tqdm(total=len(info_df['id'].unique().tolist())) as t:\n    for idd in info_df['id'].unique().tolist():\n    print(idd)\n    info_df = fetch_course_detail(idd, column_list, detail_params, info_df)\n    time.sleep(3)\n"

In [13]:
info_df.head()

,id,teachers,org_name,classfy_name,middle_classfy_name,overview
0,course-v1:ACRCEDU+ACRC01+2020_03,"신병주, 윤성은, 하재근, 오수진, 이정수",국민권익위원회 청렴연수원,인문,인문과학,"<div id=""course-info"">\n<section class=""about""..."
1,course-v1:AIIA+AIIA01+2021_T2_AIIA01,이준열,지능정보산업협회,공학,컴퓨터ㆍ통신,"<div id=""course-info"">\n<section class=""about""..."
2,course-v1:AIIA+AIIA02+2021_T2_AIIA02,,,,,"<div id=""course-info"">\n<section class=""about""..."
3,course-v1:AJOU+AJOU01+2021_T1,김경일,아주대학교,사회,사회과학,"<div id=""course-info"">\n <section class=""abou..."
4,course-v1:ANU+ANU01+2020_T3,차영욱,안동대학교,공학,컴퓨터ㆍ통신,"<div id=""course-info"">\n<section class=""about""..."


In [11]:
info_df.to_csv('course_detail.csv', index = False)